In [36]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn import metrics
print(torch.cuda.is_available())
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from vis_model_high import lung_finetune_flex,ReconstructionModel,SegmentationModel,BaseModel
from utils import *
from predict import model_predict, sr_predict_lung_segmentation, sr_predict_lung_segmentation_spot
from dataset import ViT_HER2ST, ViT_SKIN,HER2ST,LUNG,LUNG_HD
from PIL import Image
import glob
from pytorch_lightning import LightningModule
from pytorch_lightning.callbacks import ModelCheckpoint
from scipy import stats
import scanpy as sc
import cv2
import anndata
import torch.nn.functional as F
from skimage.segmentation import watershed
os.environ["TORCH_CPP_LOG_LEVEL"]="INFO"
os.environ["TORCH_DISTRIBUTED_DEBUG"] = "DETAIL"
# Check if CUDA is available
print(torch.cuda.is_available())
import numpy as np
import matplotlib.pyplot as plt
import os
import torch.optim as optim
import scipy.sparse

True
True


In [37]:
if torch.cuda.is_available():
    # Reset/release all CUDA memory
    torch.cuda.empty_cache()

    # Additional step to reset all CUDA tensors (optional)
    torch.cuda.ipc_collect()
!nvidia-smi

Thu Dec  5 05:22:35 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  |   00000000:07:00.0 Off |                    0 |
| N/A   33C    P0             65W /  400W |     425MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
import cv2
import os
import numpy as np
import scipy.sparse
from skimage.color import rgb2gray
from stardist.models import StarDist2D
from csbdeep.utils import normalize
import tifffile as tiff
# from scipy import ndimage
# from skimage.measure import regionprops
# from PIL import Image
def stardist(image_array, labels_npz_path, stardist_model="2D_versatile_he", block_size=4096*2, min_overlap=128, context=128, **kwargs):
    '''
    Segment an image with StarDist. Supports both the fluorescence and 
    H&E models. The identified object labels will be converted to a 
    sparse matrix and written to drive in ``.npz``.
    
    Input
    -----
    image_array : ``numpy array``
        Image to be segmented.
    labels_npz_path : ``filepath``
        Path to write object labels output. Can be easily loaded via 
        ``scipy.sparse.load_npz()``.
    stardist_model : ``str``, optional (default: ``"2D_versatile_he"``)
        Use ``"2D_versatile_he"`` for segmenting H&E images or 
        ``"2D_versatile_fluo"`` for segmenting GEX-derived single-channel 
        images
    block_size : ``int``, optional (default: 4096)
        StarDist ``predict_instances_big()`` input. Length of square edge 
        of the image to process as a single tile. 
    min_overlap : ``int``, optional (default: 128)
        StarDist ``predict_instances_big()`` input. Minimum overlap between 
        adjacent tiles, in each dimension.
    context : ``int``, optional (default: 128)
        StarDist ``predict_instances_big()`` input. Amount of image context 
        on all sides of a block, which is discarded.
    kwargs
        Any additional arguments to pass to StarDist. Practically most likely 
        to be ``prob_thresh`` for controlling the stringency of calling 
        objects.
    '''
    # Convert to grayscale if using the fluorescence model
    if stardist_model == "2D_versatile_fluo":
        image_array = rgb2gray(image_array)
        
    # Normalize the image
    img = normalize(image_array, 7, 99.8, axis=(0, 1))
    
    # Use pretrained stardist model
    model = StarDist2D.from_pretrained(stardist_model)
    
    # Specify axes
    if stardist_model == "2D_versatile_he":
        model_axes = "YXC"
    elif stardist_model == "2D_versatile_fluo":
        model_axes = "YX"
    
    # Run predict_instances_big() to perform automated tiling of the input
    labels, _ = model.predict_instances_big(img, axes=model_axes, 
                                            block_size=block_size, 
                                            min_overlap=min_overlap, 
                                            context=context, 
                                            **kwargs
                                           )
    
    # Store resulting labels as sparse matrix NPZ
    labels_sparse = scipy.sparse.csr_matrix(labels)
    scipy.sparse.save_npz(labels_npz_path, labels_sparse)
    print("Found " + str(len(np.unique(labels_sparse.data))) + " objects")
    return labels
def get_img(name):
    if name in ['A1', 'A2', 'A3','A4']:
        img_fold = os.path.join('/ix1/wchen/Zhaochongyue/spatial/Lung/ST/', name,
                                'outs/spatial/full_image.tif')
    elif name=='08_WT_F_S':
        name='08_WT_F-S'
        img_fold = os.path.join('/ix1/wchen/Shiyue/Projects/2023_06_Influ_Mouse_Lung_ST/RawData/Fastq/Alcorn_Visium_FFPE_Images/', name + '.TIF')
    elif name=='09_WT_F_S':
        name='09_WT_F-S'
        img_fold = os.path.join('/ix1/wchen/Shiyue/Projects/2023_06_Influ_Mouse_Lung_ST/RawData/Fastq/Alcorn_Visium_FFPE_Images/', name + '.TIF')
    else:
        img_fold = '/ix1/wchen/liutianhao/data/public_lung_spatial/CRC_10X/HD/P1_CRC/Visium_HD_Human_Colon_Cancer_P1_tissue_image.btf'
    print(os.path.exists(img_fold))
    img_color = cv2.imread(img_fold, cv2.IMREAD_UNCHANGED).astype(np.float32)
    img_color=img_color[:,:,0:3]
    #print(img_color.shape,'shape')
    # img_color = cv2.cvtColor(img_color, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB
    #print(img_color.shape,'shape1')
    return img_color
names = ['P1_CRC']
def read_bigtiff_in_chunks(file_path, chunk_size):
    with tiff.TiffFile(file_path) as tif:
        # Get image dimensions
        image_shape = tif.series[0].shape
        height, width = image_shape[:2]
        channels = 1 if len(image_shape) == 2 else image_shape[2]

        # Create an empty array to store the image
        image = np.zeros((height, width, channels), dtype=np.float32)

        # Read the image in chunks
        for i in range(0, height, chunk_size):
            for j in range(0, width, chunk_size):
                chunk = tif.asarray(key=0)[i:i+chunk_size, j:j+chunk_size, :channels]
                image[i:i+chunk_size, j:j+chunk_size, :channels] = chunk

    return image
def read_large_tiff(file_path):
    # Read the image using tifffile
    img_color = tiff.imread(file_path).astype(np.float32)

    # If the image has more than three channels, select only the first three
    if img_color.ndim == 3 and img_color.shape[2] > 3:
        img_color = img_color[:, :, 0:3]

    return img_color
chunk_size = 1024  # Example chunk size
btf_file_path='/ix1/wchen/liutianhao/data/public_lung_spatial/CRC_10X/HD/P1_CRC/Visium_HD_Human_Colon_Cancer_P1_tissue_image.btf'
# Read the BigTIFF file in chunks
image_rgb = read_large_tiff(btf_file_path)

# print(names[0])
# image_rgb = get_img(names[0])


In [ ]:
image_array = np.array(image_rgb)
np.save('/ix1/wchen/Zhaochongyue/spatial/crc_image_rgb.npy', image_array)
labels_npz_path = '/ix1/wchen/Zhaochongyue/spatial/crc_labels.npz'
labels=stardist(image_array, labels_npz_path)

In [ ]:
# Plot the labels
plt.figure(figsize=(10, 10))
plt.imshow(labels, cmap='gray')
plt.title('Segmented Labels')
plt.axis('off')
plt.show()

In [39]:
spatial_data = adata.obsm['spatial']
has_negative_values = np.any(spatial_data < 0)

print(f"Does adata.obsm['spatial'] contain negative values? {has_negative_values}")

# Optionally, print the indices and values of the negative entries for further inspection
if has_negative_values:
    negative_indices = np.where(spatial_data < 0)
    print("Indices of negative values in adata.obsm['spatial']:", negative_indices)
    print("Negative values in adata.obsm['spatial']:", spatial_data[negative_indices])

Does adata.obsm['spatial'] contain negative values? False


In [40]:
import h5py
h5_file_path = '/ix1/wchen/liutianhao/data/public_lung_spatial/CRC_10X/singlecell/HumanColonCancer_Flex_Multiplex_count_filtered_feature_bc_matrix.h5'
adata_sc = sc.read_10x_h5(h5_file_path)
meta_data = pd.read_csv('/ix1/wchen/Zhaochongyue/spatial/SingleCell_MetaData.csv')
# Filter clusters with more than 25 cells
kp_idents = meta_data['Level2'].value_counts()[meta_data['Level2'].value_counts() > 25].index
filtered_meta_data = meta_data[meta_data['Level2'].isin(kp_idents)]
# Ensure barcodes match in format between adata and meta_data
adata_barcodes = adata_sc.obs_names
meta_data_barcodes = filtered_meta_data['Barcode']
# Check the intersection of barcodes
common_barcodes = np.intersect1d(adata_barcodes, meta_data_barcodes)
# Subset AnnData to only include the common barcodes
adata_sc = adata_sc[common_barcodes]
# Also filter the metadata to include only common barcodes
filtered_meta_data = filtered_meta_data[filtered_meta_data['Barcode'].isin(common_barcodes)]
# Set the Barcode as index to facilitate merging
filtered_meta_data.set_index('Barcode', inplace=True)
# Align the indices (barcodes) of the filtered metadata with the AnnData object
adata_sc.obs = adata_sc.obs.join(filtered_meta_data, how='left')

adata_sc.var_names_make_unique()

names = ['P1_CRC']
datasets = {}
# all_centroids = []  # List to store all centroids
# all_images = []  # List to store all images
for name in names:
    datasets[name] = get_cnt(name)
    datasets[name].var_names_make_unique()
# Find common genes between single-cell and other datasets
common_genes = set(adata_sc.var_names)
for data in datasets.values():
    common_genes.intersection_update(set(data.var_names))
adata_sc = adata_sc[:, list(common_genes)]
for name in datasets:
    datasets[name] = datasets[name][:, list(common_genes)]
adata_sc = adata_sc[adata_sc.obs['QCFilter'] == "Keep"]
adata_sc2=adata_sc

sc.pp.normalize_total(adata_sc, target_sum=1e4)

sc.pp.log1p(adata_sc)


                      Barcode Patient   BC QCFilter       Level1       Level2
0  AAACAAGCAACAGCACACTTTAGG-1   P2CRC  BC1   Remove  QC_Filtered  QC_Filtered
1  AAACAAGCAACAGCTAACTTTAGG-1   P2CRC  BC1     Keep      B cells       Plasma
2  AAACAAGCAACTGTTCACTTTAGG-1   P2CRC  BC1     Keep      T cells   CD4 T cell
3  AAACAAGCAAGGCCTGACTTTAGG-1   P2CRC  BC1     Keep        Tumor    Tumor III
4  AAACAAGCACATAGTGACTTTAGG-1   P2CRC  BC1     Keep        Tumor    Tumor III


In [41]:
from collections import defaultdict, Counter
from scipy.sparse import isspmatrix
sc.tl.rank_genes_groups(adata_sc, 'Level2', method='wilcoxon')
# Define the number of top marker genes to extract
marker_genes_data = sc.get.rank_genes_groups_df(adata_sc, group=None)
logfc_threshold = 0.2
min_diff_pct = 0.2
p_val_adj_threshold = 0.05
# Filter marker genes based on thresholds
filtered_markers = marker_genes_data[
    (marker_genes_data['logfoldchanges'] > logfc_threshold) & 
    (marker_genes_data['pvals_adj'] < p_val_adj_threshold)
]
# Calculate the percentage of cells expressing each gene in the group and reference populations
group_counts = adata_sc.obs['Level2'].value_counts()
gene_names = filtered_markers['names'].unique()
# Create a DataFrame to store the fractions
fractions = pd.DataFrame(index=gene_names, columns=group_counts.index)

for group in group_counts.index:
    group_idx = adata_sc.obs['Level2'] == group
    group_data = adata_sc[group_idx, gene_names].X > 0
    group_gene_counts = group_data.sum(axis=0).A1
    fractions[group] = group_gene_counts / group_counts[group]

# Calculate the rest of the population fractions
rest_fractions = (adata_sc[:, gene_names].X > 0).sum(axis=0).A1 / adata_sc.shape[0]

# Calculate the difference in fraction of expression
filtered_markers['pct_diff'] = filtered_markers.apply(
    lambda x: fractions.loc[x['names'], x['group']] - rest_fractions[gene_names.tolist().index(x['names'])], axis=1)

# Filter based on percentage difference
filtered_markers = filtered_markers[filtered_markers['pct_diff'] > min_diff_pct]

# Select the top marker genes for each label
# Select the top marker genes for each label
top_marker_genes = defaultdict(list)
for group in adata_sc.obs['Level2'].unique():
    top_genes = filtered_markers[filtered_markers['group'] == group]['names'].tolist()
    top_marker_genes[group].extend(top_genes)

# Flatten the dictionary and count occurrences of each gene
all_genes = [gene for genes in top_marker_genes.values() for gene in genes]
gene_counts = Counter(all_genes)

# Filter out non-unique marker genes
unique_marker_genes = {group: [] for group in top_marker_genes.keys()}
for group, genes in top_marker_genes.items():
    unique_marker_genes[group] = [gene for gene in genes if gene_counts[gene] == 1]

# Use all unique marker genes instead of selecting top 5
selected_marker_genes = [gene for genes in unique_marker_genes.values() for gene in genes]

marker_dict = {
    'CD4 T cell': ['CD4'],  # List of markers for CD4+ T cells
    'CD8 Cytotoxic T cell': ['CD8A', 'PRF1', 'GZMA', 'GZMK','CXCL9', 'CXCL10','CXCL11'],  # List of markers for CD8+ T cells
    'Macrophage': ['C1QC', 'SELENOP', 'SPP1', 'STAB1','CD52','FBP1','IL1RN','FN1','LPL','CHI3L1','MMP7','CHIT1','CSF1R','MPEG1','MS4A6A','FGL2','SLC40A1','IGKC','CD209'],  # List of markers for Macrophages
    'Mature B':['JCHAIN','MZB1']
}

# Initialize the dictionary to store marker genes
small_marker_dict = defaultdict(list)

# Directly assign the predefined markers to the small_marker_dict
for cell_type, genes in marker_dict.items():
    small_marker_dict[cell_type].extend(genes)

# Extract the top 5 unique marker genes for each cell type
# unique_marker_genes = {}
for cell_type, genes in small_marker_dict.items():
    unique_marker_genes[cell_type] = list(set(genes))  # Ensure uniqueness

# Get the top 5 markers (if available)
top_5_unique_marker_genes = {group: genes[:5] for group, genes in unique_marker_genes.items()}
selected_marker_genes = [gene for genes in top_5_unique_marker_genes.values() for gene in genes if gene in common_genes]

marker_file_data = []
for group, genes in  top_5_unique_marker_genes.items():
    for gene in genes:
        marker_file_data.append({'gene': gene, 'label': group})
marker_file_df = pd.DataFrame(marker_file_data)

adata_sc_common = adata_sc[:, list(common_genes)].copy()
sc.pp.highly_variable_genes(adata_sc_common, n_top_genes=1000)
highly_variable_genes = adata_sc_common.var[adata_sc_common.var['highly_variable']].index.tolist()

# Combine selected marker genes and highly variable genes

final_gene_list = []
for gene in selected_marker_genes:
    if gene not in final_gene_list:
        final_gene_list.append(gene)

for gene in highly_variable_genes:
    if gene not in final_gene_list:
        final_gene_list.append(gene)
# If the combined list exceeds 1000 genes, trim it to 1000
if len(final_gene_list) > 1000:
    final_gene_list = final_gene_list[:1000]
with open('/ix1/wchen/Zhaochongyue/spatial/crc_hvg_list.pkl', 'wb') as f:
    pickle.dump(final_gene_list, f)
with open('/ix1/wchen/Zhaochongyue/spatial/crc_hvg_list.pkl', 'rb') as f:
    final_gene_list = pickle.load(f)
adata_sc = adata_sc2[:, final_gene_list]
data = adata_sc.X
if isspmatrix(data):  
   data = data.toarray()
data_min = data.min(axis=0)
data_max = data.max(axis=0)

adata_sc.X = (data - data_min) / (data_max - data_min+1e-12)
